# Time Series Forecasting for Infrastructure Metrics

## Objectives
- Build a predictive model using an **LSTM (Long Short-Term Memory)** neural network in PyTorch.
- Forecast future infrastructure load (e.g., CPU utilization) based on historical sequence data.
- Learn how to flag anomalies when the actual metric deviates significantly from the LSTM's prediction.

## Dataset
- We will generate a synthetic time series of CPU usage showing daily patterns and injected anomalies.

## Expected Outcome
- A trained PyTorch LSTM model capable of predicting the next hour of CPU load.
- A visualization comparing the predicted vs. actual load to spot anomalies.

## Challenge
- Can you modify the network to predict the *next 5 minutes* instead of just the next single minute?

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set seeds
torch.manual_seed(42)
np.random.seed(42)

print(f"Using device: {device}")

### 1. Generating Data
We need a time series with a clear pattern that an LSTM can learn.

In [ ]:
def generate_cpu_metrics(days=14, points_per_hour=60):
    total_points = days * 24 * points_per_hour
    t = np.linspace(0, days * 2 * np.pi, total_points)
    
    # Base daily sine wave
    base_load = 40 + 20 * np.sin(t)
    
    # Add noise
    noise = np.random.normal(0, 2, total_points)
    
    cpu_usage = base_load + noise
    
    # Inject an anomaly near the end
    cpu_usage[-200:-180] += 30
    
    return np.clip(cpu_usage, 0, 100).reshape(-1, 1)

data = generate_cpu_metrics()

plt.figure(figsize=(12, 4))
plt.plot(data[-1000:], label="CPU Usage") # Plot last ~16 hours
plt.title("CPU Usage with Anomaly at the end")
plt.legend()
plt.show()

### 2. Preprocessing for PyTorch
Neural networks train best when data is scaled between -1 and 1 or 0 and 1.

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_data = scaler.fit_transform(data)

# Function to create sequences
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 60 # Look back 60 minutes to predict the next 1 minute

X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Split into train/test (chronologically! Don't shuffle time series data initially)
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# Convert to PyTorch tensors
X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)

# Create DataLoaders
batch_size = 64
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

### 3. Defining the LSTM Model

In [ ]:
class CPUForecaster(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1, output_size=1):
        super(CPUForecaster, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, _ = self.lstm(x, (h0.detach(), c0.detach()))
        
        # Index hidden state of last time step
        out = self.fc(out[:, -1, :]) 
        return out

model = CPUForecaster().to(device)
criterion = nn.MSELoss() # Mean Squared Error is standard for regression/time-series
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### 4. Training Loop

In [ ]:
epochs = 10

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(train_loader):.4f}')

### 5. Evaluation and Anomaly Detection
When a model accurately predicts the time series, a sudden large gap between `Actual` and `Predicted` indicates an anomaly.

In [ ]:
model.eval()
with torch.no_grad():
    test_predictions = model(X_test)

# Inverse transform to get back to original CPU percentages
predictions_inv = scaler.inverse_transform(test_predictions.cpu().numpy())
y_test_inv = scaler.inverse_transform(y_test.cpu().numpy())

# Calculate Error
errors = np.abs(predictions_inv - y_test_inv)
threshold = np.mean(errors) + 3 * np.std(errors) # 3 standard deviations for anomaly threshold

# Plot the last 400 points of the test set
points_to_plot = 400
plt.figure(figsize=(15, 6))

plt.plot(y_test_inv[-points_to_plot:], label='Actual Load', alpha=0.5, color='blue')
plt.plot(predictions_inv[-points_to_plot:], label='Predicted Load', linestyle='dashed', color='green')

# Highlight points where error > threshold
recent_errors = errors[-points_to_plot:]
anomalies = np.where(recent_errors > threshold)[0]
plt.scatter(anomalies, y_test_inv[-points_to_plot:][anomalies], color='red', label='Detected Anomaly', zorder=5)

plt.title("LSTM Forecasting & Anomaly Detection")
plt.ylabel("CPU %")
plt.legend()
plt.show()